In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout,Reshape, Input, Conv1D, Dense, GlobalMaxPooling3D, BatchNormalization, Activation, Concatenate,Input
import os
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Establezco una ruta absoluta a un directorio existente de mi Google Drive
BASE_FOLDER = "/content/drive/Othercomputers/Mi portátil/Master/TFM/TFM/Datos"

In [ ]:
train_dir =BASE_FOLDER +"/Train"
val_dir = BASE_FOLDER +"/Valid"

In [ ]:
# Dimension de la matriz 3d
N = 30
BATCH_SIZE = 8
EPOCHS= 20
FILTERS_C1 = 16
FILTERS_C2 = 32
FILTERS_C3 = 64

In [ ]:
def cargar_matriz_3d_de_archivo(ruta_archivo, N):
    """
    Carga una matriz 3D de tamaño N a partir de un archivo de texto que contiene
    las coordenadas de los elementos que son 1. El resto de elementos serán 0.

    :param ruta_archivo: str - Ruta al archivo de texto.
    :param N: int - Tamaño de la matriz 3D (N x N x N).
    :return: np.ndarray - Matriz 3D de tamaño N con valores 0 y 1.
    """
    # Inicializar la matriz 3D con ceros
    matriz_3d = np.zeros((N, N, N), dtype=int)

    # Leer el archivo y procesar cada línea
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            # Separar las coordenadas (x, y, z)
            x, y, z, _ = map(int, linea.strip().split(','))

            # Asignar el valor 1 en la posición correspondiente
            if 0 <= x < N and 0 <= y < N and 0 <= z < N:
                matriz_3d[x, y, z] = 1

    return matriz_3d

In [ ]:
def cargar_etiquetas_de_archivo(archivo, N=30):
    """
    Carga las etiquetas de un archivo de texto y las devuelve en formato one-hot encoding
    para cada una de las 3 coordenadas (X, Y, Z) de los 4 cuadrantes, en total genera 12 etiquetas.

    Cada etiqueta será un array de 30 elementos con un único 1 indicando la posición correcta,
    y el resto 0.

    Si alguna coordenada es (0, 0, 0), será reemplazada por el cuarto punto calculado.

    Parámetros:
    archivo: str. Contenido del archivo con las etiquetas en formato 'x_y_z-x_y_z-x_y_z-x_y_z'.
    N: int. Tamaño de la nube de puntos (valor máximo de coordenada, por defecto 30).

    Retorna:
    etiquetas: np.array de tamaño (12, 30) (4 cuadrantes x 3 coordenadas (x, y, z)).
    """
    # Inicializar una lista para las etiquetas (4 cuadrantes x 3 coordenadas = 12)
    etiquetas = []

    # Dividir el archivo por los guiones para obtener las etiquetas de los 4 cuadrantes
    puntos_contacto = archivo.split('-')

    # Guardar los puntos ya parseados
    puntos = []

    # Parsear cada cuadrante
    for i in range(4):
        etiqueta = puntos_contacto[i]
        if etiqueta:
            # Separar las coordenadas x, y, z
            x, y, z = map(int, etiqueta.split('_'))
            puntos.append((x, y, z))
    # Convertir los puntos a etiquetas en formato one-hot encoding
    for x, y, z in puntos:
        # Generar vectores one-hot para cada coordenada
        one_hot_x = np.zeros(N)
        one_hot_x[x] = 1  # Posicionar el 1 en la coordenada x

        one_hot_y = np.zeros(N)
        one_hot_y[y] = 1  # Posicionar el 1 en la coordenada y

        one_hot_z = np.zeros(N)
        one_hot_z[z] = 1  # Posicionar el 1 en la coordenada z

        # Añadir los vectores one-hot al array de etiquetas
        etiquetas.extend([one_hot_x, one_hot_y, one_hot_z])

    # Convertir la lista de etiquetas a un array numpy
    return np.array(etiquetas)

In [ ]:
X = []
y = []

# Listar archivos en la carpeta
archivos = os.listdir(train_dir)

# Filtrar solo archivos (no directorios)
archivos = [archivo for archivo in archivos if os.path.isfile(os.path.join(train_dir, archivo))]

# Recorrer cada archivo en la carpeta
for archivo in archivos:
    #print(f"Procesando archivo: {archivo}")
    y.append(cargar_etiquetas_de_archivo(archivo,N))
    X.append(cargar_matriz_3d_de_archivo(os.path.join(train_dir, archivo), N))
    #print(matriz_etiqueta)

# Convertir X y y a arrays de numpy
X = np.array(X)  # Shape (num_samples, 30, 30, 30)
y = np.array(y)  # Shape (num_samples, 4, 3)

# Expandir la dimensión para el canal (necesario para Conv3D)
X = np.expand_dims(X, axis=-1)  # Ahora tiene forma (num_samples, 30, 30, 30, 1)
print ('X.shape:',X.shape)
print ('y.shape:',y.shape)
#y = np.expand_dims(y, axis=-1)  # Ahora tiene forma (num_samples, 30, 30, 30, 1)
# Dividir en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = [y_train[:, i, :] for i in range(12)]
y_test = [y_test[:, i, :] for i in range(12)]

X.shape: (4001, 30, 30, 30, 1)
y.shape: (4001, 12, 30)


In [ ]:
X_val = []
y_val = []

# Listar archivos en la carpeta
archivos = os.listdir(val_dir)

# Filtrar solo archivos (no directorios)
archivos = [archivo for archivo in archivos if os.path.isfile(os.path.join(val_dir, archivo))]

# Recorrer cada archivo en la carpeta
for archivo in archivos:
    #print(f"Procesando archivo: {archivo}")
    y_val.append(cargar_etiquetas_de_archivo(archivo,N))
    X_val.append(cargar_matriz_3d_de_archivo(os.path.join(val_dir, archivo), N))
    #print(matriz_etiqueta)

# Convertir X y y a arrays de numpy
X_val = np.array(X_val)  # Shape (num_samples, 30, 30, 30)
y_val = np.array(y_val)  # Shape (num_samples, 4, 3)

# Expandir la dimensión para el canal (necesario para Conv3D)
X_val = np.expand_dims(X_val, axis=-1)  # Ahora tiene forma (num_samples, 30, 30, 30, 1)
print ('X.shape:',X_val.shape)
print ('y.shape:',y_val.shape)
y_val = [y_val[:, i, :] for i in range(12)]

In [ ]:

# Definimos la métrica personalizada para medir la precisión basada en la distancia absoluta
def coordinate_accuracy(y_true, y_pred, N=30):
    # Convertir las predicciones y etiquetas a índices de la clase más probable
    y_true = tf.argmax(y_true, axis=-1)
    y_pred = tf.argmax(y_pred, axis=-1)

    # Calcular la distancia absoluta entre las predicciones y las etiquetas
    absolute_diff = tf.abs(y_true - y_pred)

    # Sumar las distancias absolutas a través de todas las coordenadas (ejes)
    total_distance = tf.reduce_sum(absolute_diff, axis=-1)

    # Escalar la distancia total dividiendo por la distancia máxima posible en cada eje
    max_distance = N * tf.cast(tf.shape(y_true)[-1], tf.float32)  # N * número de coordenadas (3 por cuadrante)

    # Escalar la distancia inversamente: cuanto menor la distancia, mayor la precisión
    accuracy = 1 - ( tf.cast(total_distance, tf.float32) / max_distance)

    # Calcular el valor promedio de precisión
    return tf.reduce_mean(accuracy)



In [ ]:
# Dimension de la matriz 3d
N = 30
BATCH_SIZE = 64
EPOCHS= 100
FILTERS_C1 = 16
FILTERS_C2 = 32
FILTERS_C3 = 64

In [ ]:
#MODELO

from tensorflow.keras.regularizers import l2

input_shape = (N, N, N, 1)  # Matriz 3D con un solo canal (grayscale)

# Definir la entrada
inputs = keras.layers.Input(shape=input_shape)

# Capas Conv3D con regularización y Dropout
x = Conv3D(filters=FILTERS_C1, kernel_size=(3, 3, 3), activation='relu', kernel_regularizer=l2(0.001))(inputs)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = Dropout(0.3)(x)  # Dropout con un 30% de apagado

x = Conv3D(filters=FILTERS_C2, kernel_size=(3, 3, 3), activation='relu', kernel_regularizer=l2(0.001))(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = Dropout(0.3)(x)

x = Conv3D(filters=FILTERS_C3, kernel_size=(3, 3, 3), activation='relu', kernel_regularizer=l2(0.001))(x)

# Aplanar la salida de las capas convolucionales
x = Flatten()(x)
x = Dropout(0.4)(x)  # Dropout después del aplanado

# Capa densa completamente conectada con regularización
x = Dense(units=256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.4)(x)

# Capa de salida: 12 salidas (una por cada coordenada) con N+1 categorías cada una
outputs = []
for _ in range(12):  # 3 coordenadas por 4 puntos
    outputs.append(Dense(units=N, activation='softmax')(x))

# Crear el modelo funcional
model = Model(inputs=inputs, outputs=outputs)



# Crear una lista de métricas, una por cada salida
metrics = [coordinate_accuracy for _ in range(12)]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)


# Mostrar resumen del modelo
model.summary()




Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 30, 30, 30, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_18 (Conv3D)        │ (None, 28, 28, 28, 16) │            448 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_12          │ (None, 14, 14, 14, 16) │              0 │ conv3d_18[0][0]        │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_24 (Dropout)      │ (None, 14, 14, 14, 16) │              0 │ max_pooling3d_12[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_19 (Conv3D)        │ (None, 12, 12, 12, 32) │         13,856 │ dropout_24[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_13          │ (None, 6, 6, 6, 32)    │              0 │ conv3d_19[0][0]        │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_25 (Dropout)      │ (None, 6, 6, 6, 32)    │              0 │ max_pooling3d_13[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_20 (Conv3D)        │ (None, 4, 4, 4, 64)    │         55,360 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_6 (Flatten)       │ (None, 4096)           │              0 │ conv3d_20[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_26 (Dropout)      │ (None, 4096)           │              0 │ flatten_6[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_78 (Dense)          │ (None, 256)            │      1,048,832 │ dropout_26[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_27 (Dropout)      │ (None, 256)            │              0 │ dense_78[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_79 (Dense)          │ (None, 30)             │          7,710 │ dropout_27[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_80 (Dense)          │ (None, 30)             │          7,710 │ dropout_27[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_81 (Dense)          │ (None, 30)             │          7,710 │ dropout_27[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_82 (Dense)          │ (None, 30)             │          7,710 │ dropout_27[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_83 (Dense)          │ (None, 30)             │          7,710 │ dropout_27[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_84 (Dense)          │ (None, 30)             │          7,710 │ dropout_27[0][0]       │
├──────────────────────

 Total params: 1,211,016 (4.62 MB)

 Trainable params: 1,211,016 (4.62 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Entrenar el modelo


model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))

Epoch 1/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 275ms/step - dense_66_coordinate_accuracy: 0.9409 - dense_67_coordinate_accuracy: 0.8207 - dense_68_coordinate_accuracy: 0.8925 - dense_69_coordinate_accuracy: 0.9519 - dense_70_coordinate_accuracy: 0.8841 - dense_71_coordinate_accuracy: 0.9213 - dense_72_coordinate_accuracy: 0.8893 - dense_73_coordinate_accuracy: 0.8424 - dense_74_coordinate_accuracy: 0.9086 - dense_75_coordinate_accuracy: 0.9207 - dense_76_coordinate_accuracy: 0.8341 - dense_77_coordinate_accuracy: 0.9219 - loss: 30.0133 - val_dense_66_coordinate_accuracy: 0.9764 - val_dense_67_coordinate_accuracy: 0.9655 - val_dense_68_coordinate_accuracy: 0.9867 - val_dense_69_coordinate_accuracy: 0.9839 - val_dense_70_coordinate_accuracy: 0.9715 - val_dense_71_coordinate_accuracy: 0.9836 - val_dense_72_coordinate_accuracy: 0.9813 - val_dense_73_coordinate_accuracy: 0.9529 - val_dense_74_coordinate_accuracy: 0.9863 - val_dense_75_coordinate_accuracy: 0.9829 - val_dense_76_coordinate_accura

In [ ]:
model.evaluate(X_val, y_val)

26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - dense_66_coordinate_accuracy: 0.9954 - dense_67_coordinate_accuracy: 0.9928 - dense_68_coordinate_accuracy: 0.9963 - dense_69_coordinate_accuracy: 0.9959 - dense_70_coordinate_accuracy: 0.9920 - dense_71_coordinate_accuracy: 0.9955 - dense_72_coordinate_accuracy: 0.9952 - dense_73_coordinate_accuracy: 0.9913 - dense_74_coordinate_accuracy: 0.9964 - dense_75_coordinate_accuracy: 0.9956 - dense_76_coordinate_accuracy: 0.9919 - dense_77_coordinate_accuracy: 0.9955 - loss: 6.3158


[6.757541179656982,
 0.9946712851524353,
 0.9928286075592041,
 0.9960737228393555,
 0.9951121211051941,
 0.9920272827148438,
 0.9949918985366821,
 0.9949919581413269,
 0.9899839758872986,
 0.9963541030883789,
 0.9939504265785217,
 0.9906651377677917,
 0.9952322244644165]

In [ ]:
model.save(BASE_FOLDER + '/FINAL-MODEL-EPOCHS' + str(EPOCHS) + '-BATCHSIZE-' + str(BATCH_SIZE) + '-FILTERS_C1-' + str(FILTERS_C1) + '-FILTERS_C2-' + str(FILTERS_C2) + '-FILTERS_C3-' + str(FILTERS_C3) + '.keras')

In [ ]:
# prompt: cargar el modelo en model desde la ruta
# Cargar el modelo en model desde la ruta
model = tf.keras.models.load_model(
    BASE_FOLDER + '/FINAL-MODEL-EPOCHS' + str(EPOCHS) + '-BATCHSIZE-' + str(BATCH_SIZE) + '-FILTERS_C1-' + str(FILTERS_C1) + '-FILTERS_C2-' + str(FILTERS_C2) + '-FILTERS_C3-' + str(FILTERS_C3) + '.keras',
    custom_objects={"coordinate_accuracy": coordinate_accuracy},
)  # Pass custom_objects to load_model

In [ ]:
def calcular_yaw_pitch_roll(puntos_contacto):
    # Extraer las posiciones de las ruedas a partir del array de entrada
    DLI =np.array(puntos_contacto[0])  # Delante izquierda
    DLD = np.array(puntos_contacto[1]) # Delante derecha
    DTI = np.array(puntos_contacto[2])  # Detrás izquierda
    DTD = np.array(puntos_contacto[3]) # Detrás derecha

    # 1. Calcular el vector forward del vehículo (dirección hacia adelante)
    frente = (DLI + DLD) / 2 - (DTI + DTD) / 2

    # 2. Calcular el vector right del vehículo (dirección lateral derecha)
    right = DLI - DLD

    # 3. Calcular el vector up del vehículo (dirección hacia arriba)
    up = np.cross(frente, right)

    # Normalizar los vectores
    frente = frente / np.linalg.norm(frente)
    right = right / np.linalg.norm(right)
    up = up / np.linalg.norm(up)

    # 4. Calcular yaw (rotación alrededor del eje y, que es el eje "up")
    yaw = np.arctan2(frente[2], frente[0])

    # 5. Calcular pitch (rotación alrededor del eje z, que es el eje "right")
    pitch = np.arcsin(-frente[1])  # El seno del ángulo de pitch está dado por la componente y del vector forward

    # 6. Calcular roll (rotación alrededor del eje x, que es el eje "forward")
    roll = np.arctan2(up[2], up[1])

    # Convertir los ángulos a grados
    yaw = np.degrees(yaw)
    pitch = np.degrees(pitch)
    roll = np.degrees(roll)

    return yaw, pitch, roll

In [ ]:
def calcular_distancia_euclidea(p1, p2):
    """Calcula la distancia euclídea entre dos puntos en 3D."""
    return np.linalg.norm(np.array(p1) - np.array(p2))

In [ ]:
def parsear_puntos_de_contacto(nombre_archivo):
    """Parsea el nombre del archivo para extraer las coordenadas reales de los puntos de contacto."""
    # Ejemplo de nombre de archivo: "17_4_1-17_17_3-2_4_1--pointCloud20241006110038303.txt..txt"
    # Partir el nombre por los guiones
    partes = nombre_archivo.split('-')[0:4]
    # Cada parte tiene las coordenadas separadas por "_", los puntos sin coordenadas serán (0, 0, 0)
    puntos_contacto = []
    for parte in partes:
        coords = [int(c) for c in parte.split('_')]
        puntos_contacto.append(tuple(coords))
    return puntos_contacto



In [ ]:

# Listar archivos en el directorio de prueba
archivos_val = os.listdir(val_dir)
archivos_val = [archivo for archivo in archivos_val if os.path.isfile(os.path.join(val_dir, archivo))]
distancia_acumulada =0
dif_yaw =0
dif_pitch =0
dif_roll =0
# Procesar cada archivo de prueba
for archivo in archivos_val:
    matriz_3d = cargar_matriz_3d_de_archivo(os.path.join(val_dir, archivo), N)

    # Expandir la dimensión para que tenga forma (1, N, N, N, 1)
    matriz_3d = np.expand_dims(matriz_3d, axis=-1)  # Añadir dimensión del canal
    matriz_3d = np.expand_dims(matriz_3d, axis=0)   # Añadir dimensión de lote

    # Realizar la predicción
    prediction = model.predict(matriz_3d)

    # Parsear puntos de contacto reales desde el nombre del archivo
    puntos_reales = parsear_puntos_de_contacto(archivo)
    yaw,pitch,roll = calcular_yaw_pitch_roll(puntos_reales)
    print(f"Archivo: {archivo}")
    distancia_total = 0
    puntos_predichos = []
    for rueda in range(4):
        # Predicción para cada coordenada de la rueda
        pred_x = np.argmax(prediction[rueda * 3])  # Coordenada X
        pred_y = np.argmax(prediction[rueda * 3 + 1])  # Coordenada Y
        pred_z = np.argmax(prediction[rueda * 3 + 2])  # Coordenada Z
        predicho = (pred_x, pred_y, pred_z)
        puntos_predichos.append(predicho)
        # Obtener el punto real de la rueda
        real = puntos_reales[rueda]

        # Calcular la distancia euclídea
        distancia = calcular_distancia_euclidea(real, predicho)
        distancia_total += distancia

        print(f"R{rueda + 1}:Real: {real}, Pred: {predicho}, Distancia: {distancia:.2f}")
    yaw_pred,pitch_pred,roll_pred = calcular_yaw_pitch_roll(puntos_predichos)
    print(f"Distancia total: {distancia_total:.2f}")
    distancia_acumulada += distancia_total
    print(f"Yaw: {yaw:.2f}, Pitch: {pitch:.2f}, Roll: {roll:.2f}")
    print(f"YawP: {yaw_pred:.2f}, PitchP: {pitch_pred:.2f}, RollP: {roll_pred:.2f}")
    print(f"Diferencia: " + str(abs(yaw-yaw_pred)) + ", " + str(abs(pitch-pitch_pred)) + ", " + str(abs(roll-roll_pred)))
    #acumular diferencias angulares
    dif_yaw += abs(yaw-yaw_pred)
    dif_pitch += abs(pitch-pitch_pred)
    dif_roll += abs(roll-roll_pred)
    print("-" * 50)
print("Distancia media: " + str(distancia_acumulada/len(archivos_val)))
print("Diferencia angular media: " + str(dif_yaw/len(archivos_val)) + ", " + str(dif_pitch/len(archivos_val)) + ", " + str(dif_roll/len(archivos_val)))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step
Archivo: 14_1_3-14_0_14-2_2_3-2_2_14-pointCloud20241007160207045.txt
R1:Real: (14, 1, 3), Pred: (14, 1, 3), Distancia: 0.00
R2:Real: (14, 0, 14), Pred: (14, 0, 14), Distancia: 0.00
R3:Real: (2, 2, 3), Pred: (2, 3, 3), Distancia: 1.00
R4:Real: (2, 2, 14), Pred: (1, 2, 14), Distancia: 1.00
Distancia total: 2.00
Yaw: 0.00, Pitch: 7.13, Roll: 5.19
YawP: 0.00, PitchP: 9.09, RollP: 5.19
Diferencia: 0.0, 1.9652605719205223, 8.881784197001252e-16
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Archivo: 15_1_3-15_2_15-2_1_3-2_2_15-pointCloud20241007160216460.txt
R1:Real: (15, 1, 3), Pred: (15, 1, 3), Distancia: 0.00
R2:Real: (15, 2, 15), Pred: (15, 2, 15), Distancia: 0.00
R3:Real: (2, 1, 3), Pred: (2, 1, 3), Distancia: 0.00
R4:Real: (2, 2, 15), Pred: (2, 2, 15), Distancia: 0.00
Distancia total: 0.00
Yaw: 0.00, Pitch: -0.00, Roll: -4.76
YawP: 0.00, PitchP: -0.00, RollP: -4.76
Diferencia: 0.0, 0.0, 0.0
---------------